# <center> **Lyft Driver Lifetime Value (LTV) Prediction and Analysis**

## <center> 🚧 Work in Progress 🚧

**Important Notice:** This notebook is currently a work in progress. It may contain incomplete sections, unpolished code, and unvalidated results. Please be aware that changes are ongoing, and the final version may differ significantly.

# <center> Project Overview and Goal

This notebook presents an analysis of Lyft datasets with the goal of predicting and analyzing the Lifetime Value of a Lyft driver. The goal of this project is to (1) **recommend a Driver's Lifetime Value** (i.e., the value of a driver to Lyft over the entire projected lifetime of a driver), and to (2) **to answer the following questions:**
   
* What are the main factors that affect a driver's lifetime value?
* What is the average projected lifetime of a driver? That is, once a driver is onboarded, how long do they typically continue driving with Lyft?
* Do all drivers act alike? Are there specific segments of drivers that generate more value for Lyft than the average driver?
* What actionable recommendations are there for the business?
  
<u>The following assumptions can be made about the Lyft rate card:<u>

* Base Fare \$2.00
* Cost per Mile \$1.15
* Cost per Minute \$0.22
* Service Fee \$1.75
* Minimum Fare \$5.00
* Maximum Fare \$400.00

# <center> Data Loading and Initial Data Exploration

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random

In [71]:
# Import "ML_Toolchest.ipynb" which contains a custom collection of user defined functions covering various steps along the ML pipeline
%run 'ML_Toolchest.ipynb'

### `Driver_ids.csv`

In [158]:
drivers = pd.read_csv('data/driver_ids.csv')
drivers.head()

,driver_id,driver_onboard_date
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00
1,007f0389f9c7b03ef97098422f902e62,2016-03-29 00:00:00
2,011e5c5dfc5c2c92501b8b24d47509bc,2016-04-05 00:00:00
3,0152a2f305e71d26cc964f8d4411add9,2016-04-23 00:00:00
4,01674381af7edd264113d4e6ed55ecda,2016-04-29 00:00:00


In [159]:
drivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   driver_id            937 non-null    object
 1   driver_onboard_date  937 non-null    object
dtypes: object(2)
memory usage: 14.8+ KB


In [160]:
drivers.nunique()

driver_id              937
driver_onboard_date     49
dtype: int64

### Ride_ids.csv

In [75]:
rides = pd.read_csv('data/ride_ids.csv')
rides.head()

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100


In [76]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193502 entries, 0 to 193501
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   driver_id        193502 non-null  object
 1   ride_id          193502 non-null  object
 2   ride_distance    193502 non-null  int64 
 3   ride_duration    193502 non-null  int64 
 4   ride_prime_time  193502 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 7.4+ MB


In [77]:
rides.describe()

,ride_distance,ride_duration,ride_prime_time
count,193502.000000,193502.000000,193502.000000
mean,6955.218266,858.966099,17.305893
std,8929.444606,571.375818,30.825800
min,-2.000000,2.000000,0.000000
25%,2459.000000,491.000000,0.000000
50%,4015.000000,727.000000,0.000000
75%,7193.000000,1069.000000,25.000000
max,724679.000000,28204.000000,500.000000


In [78]:
rides.nunique()

driver_id             937
ride_id            193502
ride_distance       28259
ride_duration        4092
ride_prime_time        12
dtype: int64

### `Ride_timestamps.csv`

In [79]:
ride_timestamps = pd.read_csv('data/ride_timestamps.csv')
ride_timestamps.head()

,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


In [80]:
ride_timestamps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970405 entries, 0 to 970404
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ride_id    970405 non-null  object
 1   event      970405 non-null  object
 2   timestamp  970404 non-null  object
dtypes: object(3)
memory usage: 22.2+ MB


In [81]:
ride_timestamps.nunique()

ride_id      194081
event             5
timestamp    865826
dtype: int64

## Data Description

There are three CSV files available with the following data:

**driver_ids.csv**

* `driver_id` - Unique identifier for a driver
* `driver_onboard_date` - Date on which driver was on-boarded

**ride_ids.csv**

* `driver_id` - Unique identifier for a driver
* `ride_id` - Unique identifier for a ride that was completed by the driver
* `ride_distance` - Ride distance in meters
* `ride_duration` - Ride duration in seconds
* `ride_prime_time` - Prime Time applied on the ride

**ride_timestamps.csv**

* `ride_id` - Unique identifier for a ride
* `event` - describes the type of event; this variable takes the following values:
* `requested_at` - passenger requested a ride
* `accepted_at` - driver accepted a passenger request
* `arrived_at` - driver arrived at pickup point
* `picked_up_at` - driver picked up the passenger
* `dropped_off_at` - driver dropped off a passenger at destination
* `timestamp` - Time of event

**Notes:**
* There are 937 unique drivers.
* There are 193,502 unique rides.
* There are timestamps for 194,081 unique rides.
* On first sight, there seems to be at least one ride associated to each driver (need to verify later).
* It appears that there are timestamps available for more rides than rides being recorded in the ride_ids table.

<u>Assumptions:<u>

* All rides in the data set occurred in San Francisco
* All timestamps in the data set are in UTC

# <center> Preliminary Data Cleaning

In [161]:
# convert columns to datetime or date format
ride_timestamps['timestamp'] = pd.to_datetime(ride_timestamps['timestamp'], utc=True)
drivers['driver_onboard_date'] = pd.to_datetime(drivers['driver_onboard_date'], utc=True)

In [82]:
# check for missing values
print(check_missing_data(drivers))
print(check_missing_data(rides))
print(check_missing_data(ride_timestamps))


Missing Data Analysis Results:

                     Missing Count  Percentage (%)
driver_id                        0             0.0
driver_onboard_date              0             0.0

Missing Data Analysis Results:

                 Missing Count  Percentage (%)
driver_id                    0             0.0
ride_id                      0             0.0
ride_distance                0             0.0
ride_duration                0             0.0
ride_prime_time              0             0.0

Missing Data Analysis Results:

           Missing Count  Percentage (%)
ride_id                0        0.000000
event                  0        0.000000
timestamp              1        0.000103


In [83]:
# inspect single timestamp record with missing data
ride_timestamps[ride_timestamps.isnull().any(axis=1)]

,ride_id,event,timestamp
434222,72f0fa0bd86800e9da5c4dced32c8735,arrived_at,NaN


In [84]:
# inspect entire ride history for record with missing timestamp
ride_timestamps[ride_timestamps['ride_id'] == "72f0fa0bd86800e9da5c4dced32c8735"]

,ride_id,event,timestamp
434220,72f0fa0bd86800e9da5c4dced32c8735,requested_at,2016-04-20 11:56:50
434221,72f0fa0bd86800e9da5c4dced32c8735,accepted_at,2016-04-20 11:57:20
434222,72f0fa0bd86800e9da5c4dced32c8735,arrived_at,NaN
434223,72f0fa0bd86800e9da5c4dced32c8735,picked_up_at,2016-04-20 12:02:20
434224,72f0fa0bd86800e9da5c4dced32c8735,dropped_off_at,2016-04-20 12:16:30


# <center> Feature Engineering

In [86]:
# set cost parameters as defined in the Lyft rate card
BASE_FARE = 2.00
COST_METER = 1.15*0.000621 #cost per mile converted to meters
#COST_METER = 1.15*0.000621371 #cost per mile converted to meters
COST_MIN = 0.22
SERVICE_FEE = 1.75
MIN_FARE = 5.00
MAX_FARE = 400.00

# calculate total cost for each ride
rides['ride_cost'] = (BASE_FARE + COST_METER*rides['ride_distance'] + COST_MIN*rides['ride_duration']/60) * (1+rides['ride_prime_time']/100) + SERVICE_FEE
rides['ride_cost_no_prime'] = BASE_FARE + COST_METER*rides['ride_distance'] + COST_MIN*rides['ride_duration']/60 + SERVICE_FEE

# set min and max fare thresholds
rides['ride_cost'] = rides['ride_cost'].apply(lambda x: max(x, MIN_FARE))
rides['ride_cost'] = rides['ride_cost'].apply(lambda x: min(x, MAX_FARE))
rides['ride_cost_no_prime'] = rides['ride_cost_no_prime'].apply(lambda x: max(x, MIN_FARE))
rides['ride_cost_no_prime'] = rides['ride_cost_no_prime'].apply(lambda x: min(x, MAX_FARE))

# calculate prime premium
#rides['ride_premium'] = rides['ride_cost'] - rides['ride_cost_no_prime']
#rides['ride_premium_perc'] = rides['ride_premium']/rides['ride_cost_no_prime']*100

rides.head()

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_cost,ride_cost_no_prime
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,8.488488,6.242326
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,9.117306,9.117306
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,8.191174,8.191174
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,77.826485,62.611188
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,17.662788,9.706394


In [131]:
ride_timestamps_pivot = ride_timestamps.pivot(index='ride_id', columns='event', values='timestamp')
ride_timestamps_pivot.reset_index(inplace=True)
ride_timestamps_pivot = ride_timestamps_pivot[['ride_id','requested_at','accepted_at','arrived_at','picked_up_at','dropped_off_at']]

ride_timestamps_pivot.head()

event,ride_id,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at
0,00003037a262d9ee40e61b5c0718f7f0,2016-06-13 09:39:19+00:00,2016-06-13 09:39:51+00:00,2016-06-13 09:44:31+00:00,2016-06-13 09:44:33+00:00,2016-06-13 10:03:05+00:00
1,00005eae40882760d675da5effb89ae3,2016-05-14 05:23:21+00:00,2016-05-14 05:23:25+00:00,2016-05-14 05:26:17+00:00,2016-05-14 05:26:18+00:00,2016-05-14 05:34:17+00:00
2,000061d42cf29f73b591041d9a1b2973,2016-05-16 15:43:09+00:00,2016-05-16 15:43:14+00:00,2016-05-16 15:47:29+00:00,2016-05-16 15:47:32+00:00,2016-05-16 15:54:18+00:00
3,00006efeb0d5e3ccad7d921ddeee9900,2016-05-11 19:29:36+00:00,2016-05-11 19:29:43+00:00,2016-05-11 19:35:11+00:00,2016-05-11 19:35:15+00:00,2016-05-11 19:40:47+00:00
4,0000d9b24d8ccdd991b76258e616fa01,2016-04-26 18:11:38+00:00,2016-04-26 18:12:15+00:00,2016-04-26 18:16:52+00:00,2016-04-26 18:16:55+00:00,2016-04-26 18:24:14+00:00


In [132]:
ride_timestamps_pivot.shape

(194081, 6)

In [133]:
ride_timestamps_pivot.nunique()

event
ride_id           194081
requested_at      190434
accepted_at       190426
arrived_at        190526
picked_up_at      190524
dropped_off_at    190468
dtype: int64

In [134]:
ride_timestamps_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194081 entries, 0 to 194080
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   ride_id         194081 non-null  object             
 1   requested_at    194081 non-null  datetime64[ns, UTC]
 2   accepted_at     194081 non-null  datetime64[ns, UTC]
 3   arrived_at      194080 non-null  datetime64[ns, UTC]
 4   picked_up_at    194081 non-null  datetime64[ns, UTC]
 5   dropped_off_at  194081 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](5), object(1)
memory usage: 8.9+ MB


## Joining `ride_ids` and `ride_timestamps` data

In [178]:
rides_full = pd.merge(rides, ride_timestamps_pivot, on='ride_id', how='left')
rides_full.head()

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_cost,ride_cost_no_prime,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,8.488488,6.242326,2016-04-23 02:13:50+00:00,2016-04-23 02:14:15+00:00,2016-04-23 02:16:36+00:00,2016-04-23 02:16:40+00:00,2016-04-23 02:22:07+00:00
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,9.117306,9.117306,2016-03-29 19:00:49+00:00,2016-03-29 19:00:52+00:00,2016-03-29 19:03:57+00:00,2016-03-29 19:04:01+00:00,2016-03-29 19:17:30+00:00
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,8.191174,8.191174,2016-06-21 11:56:31+00:00,2016-06-21 11:56:39+00:00,2016-06-21 12:01:32+00:00,2016-06-21 12:01:35+00:00,2016-06-21 12:11:07+00:00
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,77.826485,62.611188,2016-05-19 09:15:29+00:00,2016-05-19 09:15:33+00:00,2016-05-19 09:18:20+00:00,2016-05-19 09:18:20+00:00,2016-05-19 10:13:58+00:00
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,17.662788,9.706394,2016-04-20 22:05:30+00:00,2016-04-20 22:05:32+00:00,2016-04-20 22:07:03+00:00,2016-04-20 22:07:02+00:00,2016-04-20 22:20:45+00:00


In [179]:
rides_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193502 entries, 0 to 193501
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   driver_id           193502 non-null  object             
 1   ride_id             193502 non-null  object             
 2   ride_distance       193502 non-null  int64              
 3   ride_duration       193502 non-null  int64              
 4   ride_prime_time     193502 non-null  int64              
 5   ride_cost           193502 non-null  float64            
 6   ride_cost_no_prime  193502 non-null  float64            
 7   requested_at        184819 non-null  datetime64[ns, UTC]
 8   accepted_at         184819 non-null  datetime64[ns, UTC]
 9   arrived_at          184818 non-null  datetime64[ns, UTC]
 10  picked_up_at        184819 non-null  datetime64[ns, UTC]
 11  dropped_off_at      184819 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns

In [180]:
# time duration between ride events
rides_full['acceptance_time'] = (rides_full['accepted_at'] - rides_full['requested_at']).dt.total_seconds()
rides_full['arrival_time'] = (rides_full['arrived_at'] - rides_full['accepted_at']).dt.total_seconds()
rides_full['pickup_time'] = (rides_full['picked_up_at'] - rides_full['arrived_at']).dt.total_seconds()
rides_full['ride_time'] = (rides_full['dropped_off_at'] - rides_full['picked_up_at']).dt.total_seconds()

rides_full['ride_date'] = rides_full['requested_at'].dt.date
# rides_full['ride_month'] = 
# rides_full['ride_season'] = 
# rides_full['ride_rush_hour'] = 
rides_full['ride_weekday'] = rides_full['requested_at'].dt.day_name()

In [181]:
# extract hour from ride timestamp
rides_full['hour'] = rides_full['requested_at'].dt.hour

# define conditions and corresponding choices
conditions = [
    (rides_full['hour'] >= 5) & (rides_full['hour'] < 8),
    (rides_full['hour'] >= 8) & (rides_full['hour'] < 12),
    (rides_full['hour'] >= 12) & (rides_full['hour'] < 13),
    (rides_full['hour'] >= 13) & (rides_full['hour'] < 17),
    (rides_full['hour'] >= 17) & (rides_full['hour'] < 20),
    (rides_full['hour'] >= 20) & (rides_full['hour'] < 24),
    (rides_full['hour'] >= 0) & (rides_full['hour'] < 5)]

choices = [
    'Early Morning', 
    'Morning', 
    'Midday', 
    'Afternoon', 
    'Evening', 
    'Night', 
    'Late Night']

# Apply conditions using np.select
rides_full['part_of_day'] = np.select(conditions, choices, default='Unknown')

In [182]:
rides_full.head()

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_cost,ride_cost_no_prime,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at,acceptance_time,arrival_time,pickup_time,ride_time,ride_date,ride_weekday,hour,part_of_day
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,8.488488,6.242326,2016-04-23 02:13:50+00:00,2016-04-23 02:14:15+00:00,2016-04-23 02:16:36+00:00,2016-04-23 02:16:40+00:00,2016-04-23 02:22:07+00:00,25.0,141.0,4.0,327.0,2016-04-23,Saturday,2.0,Late Night
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,9.117306,9.117306,2016-03-29 19:00:49+00:00,2016-03-29 19:00:52+00:00,2016-03-29 19:03:57+00:00,2016-03-29 19:04:01+00:00,2016-03-29 19:17:30+00:00,3.0,185.0,4.0,809.0,2016-03-29,Tuesday,19.0,Evening
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,8.191174,8.191174,2016-06-21 11:56:31+00:00,2016-06-21 11:56:39+00:00,2016-06-21 12:01:32+00:00,2016-06-21 12:01:35+00:00,2016-06-21 12:11:07+00:00,8.0,293.0,3.0,572.0,2016-06-21,Tuesday,11.0,Morning
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,77.826485,62.611188,2016-05-19 09:15:29+00:00,2016-05-19 09:15:33+00:00,2016-05-19 09:18:20+00:00,2016-05-19 09:18:20+00:00,2016-05-19 10:13:58+00:00,4.0,167.0,0.0,3338.0,2016-05-19,Thursday,9.0,Morning
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,17.662788,9.706394,2016-04-20 22:05:30+00:00,2016-04-20 22:05:32+00:00,2016-04-20 22:07:03+00:00,2016-04-20 22:07:02+00:00,2016-04-20 22:20:45+00:00,2.0,91.0,-1.0,823.0,2016-04-20,Wednesday,22.0,Night


In [175]:
# calculate the driver tenure
todays_date = pd.Timestamp.now(tz='UTC').normalize()
drivers['tenure_days'] = (todays_date - drivers['driver_onboard_date']).dt.days

In [177]:
# aggregate ride metrics for each driver
driver_ride_metrics = rides_full.groupby('driver_id').agg(
    total_rides=('ride_id', 'count'),
    total_distance=('ride_distance', 'sum'),
    total_duration=('ride_duration', 'sum'),
    average_ride_distance=('ride_distance', 'mean'),
    average_ride_duration=('ride_duration', 'mean'),
    prime_time_ride_ratio=('ride_prime_time', lambda x: (x > 0).mean()),
    average_acceptance_time=('acceptance_time', 'mean'),
    average_arrival_time=('arrival_time', 'mean'),
    average_pickup_time=('pickup_time', 'mean'),
    average_ride_time=('ride_time', 'mean')
).reset_index()

# merge the ride metrics with driver data
drivers_full = drivers.merge(driver_ride_metrics, on='driver_id')
drivers_full.head()

,driver_id,driver_onboard_date,tenure,tenure_days,total_rides,total_distance,total_duration,average_ride_distance,average_ride_duration,prime_time_ride_ratio,average_acceptance_time,average_arrival_time,average_pickup_time,average_ride_time
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00+00:00,2985,2985,277,1740287,221238,6282.624549,798.693141,0.397112,8.407942,172.989170,5.231047,798.693141
1,007f0389f9c7b03ef97098422f902e62,2016-03-29 00:00:00+00:00,2985,2985,31,117531,20497,3791.322581,661.193548,0.387097,11.838710,177.161290,4.870968,661.193548
2,011e5c5dfc5c2c92501b8b24d47509bc,2016-04-05 00:00:00+00:00,2978,2978,34,269653,29205,7930.970588,858.970588,0.470588,7.294118,223.235294,4.088235,858.970588
3,0152a2f305e71d26cc964f8d4411add9,2016-04-23 00:00:00+00:00,2960,2960,191,1471239,174521,7702.821990,913.722513,0.251309,14.198953,234.528796,5.513089,913.722513
4,01674381af7edd264113d4e6ed55ecda,2016-04-29 00:00:00+00:00,2954,2954,375,3123644,357443,8329.717333,953.181333,0.264000,12.528000,216.488000,8.208000,953.181333
